In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pgscen.engine import GeminiEngine, GeminiModel
from pgscen.utils.data_utils import split_actuals_hist_future, split_forecasts_hist_future
from pathlib import Path
from pgscen.pca import PCAGeminiEngine, PCAGeminiModel, standardize
from pgscen.utils.solar_utils import get_yearly_date_range
from pgscen.utils.r_utils import gemini, graphical_lasso, qdist
import random
from typing import List, Dict, Tuple, Iterable, Optional
from scipy.linalg import sqrtm
from scipy.stats import norm
from scipy.linalg import null_space
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from astral.sun import sun
from astral import LocationInfo 

In [ ]:
def load_wind_data(data_path):
    wind_site_actual_df = pd.read_csv(
        Path(data_path, 'Wind', 'NREL', 'Actual',
             'wind_actual_1h_site_2017_2018_utc.csv'),
        parse_dates=['Time'], index_col='Time'
        )

    wind_site_forecast_df = pd.read_csv(
        Path(data_path, 'Wind', 'NREL', 'Day-ahead', 'PF',
             'wind_day_ahead_forecast_site_2018_utc.csv'),
        parse_dates=['Issue_time', 'Forecast_time']
        )

    wind_meta_df = pd.read_excel(
        Path(data_path, 'MetaData', 'wind_meta.xlsx'))

    return wind_site_actual_df, wind_site_forecast_df, wind_meta_df

In [ ]:
def load_solar_data(data_path):
    solar_site_actual_df = pd.read_csv(
        Path(data_path, 'Solar', 'NREL', 'Actual',
             'solar_actual_1h_site_2017_2018_utc.csv'),
        parse_dates=['Time'], index_col='Time'
        )

    solar_site_forecast_df = pd.read_csv(
        Path(data_path, 'Solar', 'NREL', 'Day-ahead',
             'solar_day_ahead_forecast_site_2017_2018_utc.csv'),
        parse_dates=['Issue_time', 'Forecast_time']
        )

    solar_meta_df = pd.read_excel(
        Path(data_path, 'MetaData', 'solar_meta.xlsx'))

    return solar_site_actual_df, solar_site_forecast_df, solar_meta_df

In [ ]:
def load_load_data(data_path):
    load_zone_actual_df = pd.read_csv(
        Path(data_path, 'Load', 'ERCOT', 'Actual',
             'load_actual_1h_zone_2017_2018_utc.csv'),
        parse_dates=['Time'], index_col='Time'
        )

    load_zone_forecast_df = pd.read_csv(
        Path(data_path, 'Load', 'ERCOT', 'Day-ahead',
             'load_day_ahead_forecast_zone_2017_2018_utc.csv'),
        parse_dates=['Issue_time', 'Forecast_time']
        )

    return load_zone_actual_df, load_zone_forecast_df


In [ ]:
scen_start_time = pd.to_datetime('2018-08-01 06:00:00',utc=True)
nscen = 1000

In [ ]:
load_zone_actual_df, load_zone_forecast_df = load_load_data('../data/')
solar_site_actual_df, solar_site_forecast_df, solar_meta_df = load_solar_data('../data/')
wind_site_actual_df, wind_site_forecast_df, wind_meta_df = load_wind_data('../data/')

In [ ]:
scen_timesteps = pd.date_range(start=scen_start_time,periods=24, freq='H')

(load_zone_actual_hists,
            load_zone_actual_futures) = split_actuals_hist_future(
                    load_zone_actual_df, scen_timesteps, in_sample=True)

(load_zone_forecast_hists,
            load_zone_forecast_futures) = split_forecasts_hist_future(
                    load_zone_forecast_df, scen_timesteps, in_sample=True)


(solar_site_actual_hists,
            solar_site_actual_futures) = split_actuals_hist_future(
                    solar_site_actual_df, scen_timesteps, in_sample=True)

(solar_site_forecast_hists,
            solar_site_forecast_futures) = split_forecasts_hist_future(
                    solar_site_forecast_df, scen_timesteps, in_sample=True)

(wind_site_actual_hists,
            wind_site_actual_futures) = split_actuals_hist_future(
                    wind_site_actual_df, scen_timesteps, in_sample=True)

(wind_site_forecast_hists,
            wind_site_forecast_futures) = split_forecasts_hist_future(
                    wind_site_forecast_df, scen_timesteps, in_sample=True)

In [ ]:
pge = PCAGeminiEngine(solar_site_actual_hists, 
                      solar_site_forecast_hists, 
                      scen_start_time, 
                      solar_meta_df)

# Fit solar model

In [ ]:
dist = pge.asset_distance().values

pge.fit(num_of_components=12, asset_rho=dist/(10*dist.max()), horizon_rho=5e-2)

In [ ]:
pge.create_scenario(nscen, solar_site_forecast_futures)

In [ ]:
# pge.model.asset_cov.to_csv('/Users/xy3134/Research/PERFORM/notebooks/visualization/ERCOT/data/solar_asset_cov.csv',
#                           index=False)

In [ ]:
pge.write_to_csv('/Users/xy3134/Research/PERFORM/Data/scenario/test/', 
                 {'solar':solar_site_actual_futures})

# Fit load and solar joint model

In [ ]:
dist = pge.asset_distance().values

pge.fit_load_solar_joint_model(num_of_components=12, asset_rho=dist/(10*dist.max()), 
                               horizon_rho=5e-2, load_hist_actual_df=load_zone_actual_hists, 
                               load_hist_forecast_df=load_zone_forecast_hists)

In [ ]:
pge.create_load_solar_joint_scenario(nscen, load_zone_forecast_futures, solar_site_forecast_futures)

In [ ]:
pge.write_to_csv('/Users/xy3134/Research/PERFORM/Data/scenario/test/', 
                 {'load':load_zone_actual_futures, 'solar':solar_site_actual_futures})

In [ ]:
pge.joint_md.asset_cov.to_csv('/Users/xy3134/Research/PERFORM/notebooks/visualization/ERCOT/data/joint_load_solar_cov.csv',
                             index=False)

# Wind

In [ ]:
ge = GeminiEngine(wind_site_actual_hists, wind_site_forecast_hists, 
                 scen_start_time, wind_meta_df, 'wind')

In [ ]:
dist = ge.asset_distance().values

ge.fit(asset_rho=dist/(10*dist.max()), horizon_rho=5e-2)

In [ ]:
ge.create_scenario(nscen, wind_site_forecast_futures)

In [ ]:
ge.write_to_csv('/Users/xy3134/Research/PERFORM/Data/scenario/test/', {'wind':wind_site_actual_futures})

In [ ]:
ge.model.asset_cov.to_csv('/Users/xy3134/Research/PERFORM/notebooks/visualization/ERCOT/data/wind_asset_cov.csv',
                         index=False)